<a href="https://colab.research.google.com/github/Bhavi-Khator/Myntra_Hackerramp-21/blob/bhavi/LIP_hackerRamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import OrderedDict
import torch
import torch.nn as nn
import torch.nn.functional as F
import threading
from torchvision.models.densenet import densenet121, densenet161
import os
import argparse
import logging
import numpy as np
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
from torchvision import transforms
import time
import pandas as pd
import urllib.request


In [ ]:
class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', nn.BatchNorm2d(num_input_features)),
        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv2d(num_input_features, bn_size *
                                            growth_rate, kernel_size=1, stride=1, bias=False)),
        self.add_module('norm2', nn.BatchNorm2d(bn_size * growth_rate)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                                            kernel_size=3, stride=1, padding=1, bias=False)),
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)

In [ ]:
class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate, growth_rate, bn_size, drop_rate)
            self.add_module('denselayer%d' % (i + 1), layer)

In [ ]:
class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features, downsample=True):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        if downsample:
            self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))
        else:
            self.add_module('pool', nn.AvgPool2d(kernel_size=1, stride=1))  # compatibility hack

In [ ]:
class DenseNet(nn.Module):
    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0, pretrained=True):

        super(DenseNet, self).__init__()

        # First convolution
        self.start_features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(3, num_init_features, kernel_size=7, stride=2, padding=3, bias=False)),
            ('norm0', nn.BatchNorm2d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
        ]))

        # Each denseblock
        num_features = num_init_features

        init_weights = list(densenet121(pretrained=True).features.children())
        start = 0
        for i, c in enumerate(self.start_features.children()):
            if pretrained:
                c.load_state_dict(init_weights[i].state_dict())
            start += 1
        self.blocks = nn.ModuleList()
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers=num_layers, num_input_features=num_features,
                                bn_size=bn_size, growth_rate=growth_rate, drop_rate=drop_rate)
            if pretrained:
                block.load_state_dict(init_weights[start].state_dict())
            start += 1
            self.blocks.append(block)
            setattr(self, 'denseblock%d' % (i + 1), block)

            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                downsample = i < 1
                trans = _Transition(num_input_features=num_features, num_output_features=num_features // 2,
                                    downsample=downsample)
                if pretrained:
                    trans.load_state_dict(init_weights[start].state_dict())
                start += 1
                self.blocks.append(trans)
                setattr(self, 'transition%d' % (i + 1), trans)
                num_features = num_features // 2

    def forward(self, x):
        out = self.start_features(x)
        deep_features = None
        for i, block in enumerate(self.blocks):
            out = block(out)
            if i == 5:
                deep_features = out

        return out, deep_features

In [ ]:
def densenet(pretrained=True):
    return DenseNet(pretrained=pretrained)

In [ ]:
class PSPModule(nn.Module):
    def __init__(self, features, out_features=1024, sizes=(1, 2, 3, 6)):
        super().__init__()
        self.stages = []
        self.stages = nn.ModuleList([self._make_stage(features, size) for size in sizes])
        self.bottleneck = nn.Conv2d(features * (len(sizes) + 1), out_features, kernel_size=1)
        self.relu = nn.ReLU()

    def _make_stage(self, features, size):
        prior = nn.AdaptiveAvgPool2d(output_size=(size, size))
        conv = nn.Conv2d(features, features, kernel_size=1, bias=False)
        return nn.Sequential(prior, conv)

    def forward(self, feats):
        h, w = feats.size(2), feats.size(3)
        priors = [F.interpolate(input=stage(feats), size=(h, w), mode='bilinear', align_corners=False) for stage in self.stages] + [feats]
        bottle = self.bottleneck(torch.cat(priors, 1))
        return self.relu(bottle)

In [ ]:
class PSPUpsample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.PReLU()
        )

    def forward(self, x):
        h, w = 2 * x.size(2), 2 * x.size(3)
        p = F.interpolate(input=x, size=(h, w), mode='bilinear', align_corners=False)
        return self.conv(p)

In [ ]:
class PSPNet(nn.Module):
    def __init__(self, n_classes=20, sizes=(1, 2, 3, 6), psp_size=2048, deep_features_size=1024, backend='resnet34',
                 pretrained=True):
        super().__init__()
        self.feats = densenet(pretrained)
        self.psp = PSPModule(psp_size, 1024, sizes)
        self.drop_1 = nn.Dropout2d(p=0.3)

        self.up_1 = PSPUpsample(1024, 256)
        self.up_2 = PSPUpsample(256, 64)
        self.up_3 = PSPUpsample(64, 64)

        self.drop_2 = nn.Dropout2d(p=0.15)
        self.final = nn.Sequential(
            nn.Conv2d(64, n_classes, kernel_size=1),
            nn.LogSoftmax(dim=1)
        )

        self.classifier = nn.Sequential(
            nn.Linear(deep_features_size, 256),
            nn.ReLU(),
            nn.Linear(256, n_classes)
        )

    def forward(self, x):
        f, class_f = self.feats(x) 
        p = self.psp(f)
        p = self.drop_1(p)

        p = self.up_1(p)
        p = self.drop_2(p)

        p = self.up_2(p)
        p = self.drop_2(p)

        p = self.up_3(p)
        p = self.drop_2(p)

        auxiliary = F.adaptive_max_pool2d(input=class_f, output_size=(1, 1)).view(-1, class_f.size(1))

        return self.final(p), self.classifier(auxiliary)

In [ ]:
models = {
    'densenet': lambda: PSPNet(sizes=(1, 2, 3, 6), psp_size=1024, deep_features_size=512, backend='densenet')
}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
if torch.cuda.is_available():
    map_location=lambda storage, loc: storage.cuda()
else:
    map_location='cpu'

cpu


In [ ]:
def build_network(snapshot, backend):
    epoch = 0
    backend = backend.lower()
    net = models[backend]()
    net = nn.DataParallel(net)
    if snapshot is not None:
        _, epoch = os.path.basename(snapshot).split('_')
        if not epoch == 'last':
            epoch = int(epoch)
        net.load_state_dict(torch.load(snapshot, map_location=map_location))
        logging.info("Snapshot for epoch {} loaded from {}".format(epoch, snapshot))
    net = net.to(device)
    return net, epoch

In [ ]:
def get_transform():
    transform_image_list = [
        transforms.Resize((256, 256), 3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
    return transforms.Compose(transform_image_list)

In [ ]:
class saveThread (threading.Thread):
   def __init__(self, dirctry, fileName, port):
      threading.Thread.__init__(self)
      self.dirctry = dirctry
      self.fileName = fileName
      self.port = port
   def run(self):
      save(self.dirctry, self.fileName, self.port)

In [ ]:
def save(dirctry, fileName, port):
    path = dirctry + '/'+fileName
    print(path)
    print(fileName)
    service = BlobServiceClient.from_connection_string(conn_str=connection_string)
    blob_client = service.get_blob_client(container='imagegen', blob='BodyMask/'+fileName+'.jpg')
    try:
        with open(fileName+'bodymask.jpg', 'rb') as data:  
            blob_client.upload_blob(data,overwrite=True)
    except Exception as e:
        print(e)
        return
    blob_client = service.get_blob_client(container='imagegen', blob='ClothMask/'+fileName+'.jpg')
    try:
        with open(fileName+'clothmask.jpg', 'rb') as data:  
            blob_client.upload_blob(data,overwrite=True)
    except Exception as e:
        print(e)
        return
    blob_client = service.get_blob_client(container='imagegen', blob='HeadMask/'+fileName+'.jpg')
    try:
        with open(fileName+'headmask.jpg', 'rb') as data:  
            blob_client.upload_blob(data,overwrite=True)
    except Exception as e:
        print(e)
        return
    blob_client = service.get_blob_client(container='imagegen', blob='Head/'+fileName+'.jpg')
    try:
        with open(fileName+'head.jpg', 'rb') as data:  
            blob_client.upload_blob(data,overwrite=True)
    except Exception as e:
        print(e)
        return
    blob_client = service.get_blob_client(container='imagegen', blob='Cloth/'+fileName+'.jpg')
    try:
        with open(fileName+'cloth.jpg', 'rb') as data:  
            blob_client.upload_blob(data,overwrite=True)
    except Exception as e:
        print(e)
        return
    print('uploaded')
    os.remove(fileName+'cloth.jpg')
    os.remove(fileName+'headmask.jpg')
    os.remove(fileName+'clothmask.jpg')
    os.remove(fileName+'bodymask.jpg')
    os.remove(fileName+'head.jpg')
    os.remove(fileName+'temp.jpg')

In [ ]:
def show_image(img, pred, temp_image, image_url):
    # fig, axes = plt.subplots(1, 2)
    # ax0, ax1 = axes
    # ax0.get_xaxis().set_ticks([])
    # ax0.get_yaxis().set_ticks([])
    # ax1.get_xaxis().set_ticks([])
    # ax1.get_yaxis().set_ticks([])

    classes = np.array(('Background',  # always index 0
                        'Hat', 'Hair', 'Glove', 'Sunglasses',
                        'UpperClothes', 'Dress', 'Coat', 'Socks',
                        'Pants', 'Jumpsuits', 'Scarf', 'Skirt', 
                        'Face', 'Left-arm', 'Right-arm', 'Left-leg',
                        'Right-leg', 'Left-shoe', 'Right-shoe'))
    colormap = [(0, 0, 0),
                (1, 0.25, 0), (0, 0.25, 0), (0.5, 0, 0.25), (1, 1, 1),
                (1, 0.75, 0), (0, 0, 0.5), (0.5, 0.25, 0), (0.75, 0, 0.25),
                (1, 0, 0.25), (0, 0.5, 0), (0.5, 0.5, 0), (0.25, 0, 0.5),
                (1, 0, 0.75), (0, 0.5, 0.5), (0.25, 0.5, 0.5), (1, 0, 0),
                (1, 0.25, 0), (0, 0.75, 0), (0.5, 0.75, 0), ]
    cmap = matplotlib.colors.ListedColormap(colormap)
    bounds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
    norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)

    h, w, _ = pred.shape

    def denormalize(img, mean, std):
        c, _, _ = img.shape
        for idx in range(c):
            img[idx, :, :] = img[idx, :, :] * std[idx] + mean[idx]
        return img

    img = denormalize(img.cpu().numpy(), [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    img = img.transpose(1, 2, 0).reshape((h, w, 3))
    pred = pred.reshape((h, w))
    print(pred.shape)
    
    masks = []
    for i in range(len(classes)):
      masks.append((pred == i))

    clothmask = (pred==5.0) | (pred==15.0)| (pred==14.0)
    #headmask = (pred == 1.0) | (pred == 2.0) | (pred == 4.0) | (pred == 13.0)
    dress = (pred == 6.0)
    pants=(pred==9.0)
    jumpsuits=(pred==10.0)
    Skirt=(pred==12.0)   

    # bodymask = (pred != 2.0) & (pred != 1.0) & (pred != 4.0) & (pred != 13.0) & (pred != 0.0)

    orig_image = Image.open(temp_image)
    sec_image = Image.new("RGB", orig_image.size, (255,255,255))
    
    clothmask = Image.fromarray(clothmask).resize(orig_image.size)
    print(orig_image.size)
    print(clothmask.size)
    clothmask.save("clothmask.jpg")
    cloth = Image.composite(orig_image, sec_image, clothmask)
    cloth.save('/content/drive/My Drive/Myntra_hackerRamp/cloth.jpg') 
    

In [ ]:
def main(image_url):
    backend='densenet'
    models_path='/content/drive/My Drive/Myntra_hackerRamp'
    # csv_file='images_url_temp.csv'
    # --------------- model --------------- #
    snapshot = os.path.join(models_path, backend, 'PSPNet_last')
    net, starting_epoch = build_network(snapshot, backend)
    net.eval()

    # ------------ load image ------------ #
    # df = pd.read_csv(csv_file)
    # images = df['image_url'].to_numpy()
    data_transform = get_transform()
    tic_whole = time.time()
    print('start processing...')
    saving_thread=0
    print(image_url)
    temp_image = 'image.jpg'
    urllib.request.urlretrieve(str(image_url), temp_image)
    img = Image.open(temp_image,'r')
    img = data_transform(img)
    img = img.to(device)
    with torch.no_grad():
        tic = time.time()
        pred, _ = net(img.unsqueeze(dim=0))
        toc = time.time()
        print('Prediction in %.5f seconds' % (toc - tic))
        pred = pred.squeeze(dim=0)
        pred = pred.cpu().numpy().transpose(1, 2, 0)
        pred = np.asarray(np.argmax(pred, axis=2), dtype=np.uint8).reshape((256, 256, 1))
        show_image(img, pred, temp_image, image_url)
    toc_whole = time.time()
    print ('Time Elapsed is %.5f' % (toc_whole - tic_whole))
        
        
    # generate image with body parts
    # saving_thread.join()
    print ('processing time is %.5f' % (toc_whole - tic_whole))

In [ ]:
main('https://i.ibb.co/7SZnVkz/quiero.jpg')

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


start processing...
https://i.ibb.co/7SZnVkz/quiero.jpg
Prediction in 1.26409 seconds
(256, 256)
(526, 700)
(526, 700)
Time Elapsed is 2.85353
processing time is 2.85353


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive
